#### <font color=red>采用了BIEO_50pad的形式，看看效果，与BIO_40做对比

In [1]:
import numpy as np
import pandas as pd
import json,os,gc,sys,time
from datetime import datetime
import tensorflow as tf
from tensorflow.python.util import compat
from keras.models import Sequential,load_model
from keras.preprocessing.sequence import pad_sequences
from keras import backend as K
from keras_contrib.losses import crf_loss
from keras_contrib.metrics import crf_viterbi_accuracy
from keras.layers import Embedding, Bidirectional, LSTM
from keras_contrib.layers import CRF
from sklearn.model_selection import train_test_split,KFold
from keras.callbacks import EarlyStopping,ModelCheckpoint
import matplotlib as mpl
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

/home/mdm_user/venvs/tensorflow1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:458: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/mdm_user/venvs/tensorflow1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:459: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/mdm_user/venvs/tensorflow1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:460: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/mdm_user/venvs/tensorflow1/lib/python3.6/site-

In [2]:
class Word2Id:
    def __init__(self,vocab_file_path,train_data_path,labels):
        self.vocab_file_path = vocab_file_path
        self.train_data_path = train_data_path
        self.labels = labels

    def gen_save(self):
        all_char = []
        print('start geting word2id......')
        file=pd.read_csv(self.train_data_path)
        data=file[['desc']].drop_duplicates().reset_index(drop=True)
        for i in range(len(data)-1):
            s=[x for x in data['desc'][i]]
            for j in s:
                if j not in all_char:
                    all_char.extend(j)
        chars=set(all_char)
        word2id = {char: id_ + 1 for id_, char in enumerate(chars)}
        word2id["unk"] = 0
        print('train+valid一共涉及多少字符：',len(word2id))
        gc.collect()
        
        with open(self.vocab_file_path, "wb") as f:
            f.write(json.dumps(word2id, ensure_ascii=False).encode('utf-8'))

    def load(self):
        label2id = {label: id_ for id_, label in enumerate(self.labels)} #编码
        print(label2id)
        return json.load(open(self.vocab_file_path, 'r',encoding='utf-8')),label2id

    
class DataProcess:
    def __init__(self,dt,vocab,label2id,pad_embedding,is_train,del_last):
        if 'csv' in dt:
            self.data=pd.read_csv(dt)
        else:
            self.data=dt

        self.vocab=vocab
        self.label2id=label2id
        self.is_train=is_train
        self.del_last=del_last
        self.pad_embedding=pad_embedding
        self.process_data = self.process_data()
        
    def process_data(self):
        if self.is_train==1:
            dtt=self.data[['desc','final_ner']].values
            if self.del_last==1:  #当del_last为1的时候，代表的是处理整体数据的时候，如果分批处理则用del_last=0
                leng=len(dtt)-1
            else:
                leng=len(dtt)
                
            train_data=[]
            for i in range(leng): #等同于train_data.pop()去掉最后一个样本[[],[],[]]
                lst=[]
                d1=dtt[i][0]
                d2=dtt[i][1].split(',')
                for j in range(len(d1)):
                    w1=d1[j]
                    w2=d2[j]
                    lst.append([w1,w2])
                train_data.append(lst) #[[['word1','ner1'],['word2','ner2'],[]],[[],[],[]],[[],[],[]],[[],[],[]]]
            return train_data
        else:
            dtt=self.data[['desc']].values
            print(dtt.shape)
            train_data=[]
            for i in range(len(dtt)):
                lst=[]
                d1=dtt[i][0]
                for j in range(len(d1)):
                    w1=d1[j]
                    lst.append([w1])
                train_data.append(lst)
            return train_data

    def generate_data(self):
        if self.is_train==1:
            char_data_sen = [[token[0] for token in i] for i in self.process_data]
            label_sen = [[token[1] for token in i] for i in self.process_data]
        
            sen2id = [[self.vocab.get(char, 0) for char in sen] for sen in char_data_sen]
            lab_sen2id = [[self.label2id.get(lab, 0) for lab in sen] for sen in label_sen]
            
            sen_pad = pad_sequences(sen2id, self.pad_embedding,padding='post',truncating='post')
            lab_pad = pad_sequences(lab_sen2id, self.pad_embedding, value=-1,padding='post',truncating='post')
            lab_pad = np.expand_dims(lab_pad, 2)
            return sen_pad, lab_pad
        
        else:
            char_data_sen = [[token[0] for token in i] for i in self.process_data]
            sen2id = [[self.vocab.get(char, 0) for char in sen] for sen in char_data_sen]
            sen_pad = pad_sequences(sen2id,self.pad_embedding)
            return sen_pad

In [3]:
class CallBack(tf.contrib.keras.callbacks.Callback):
    def __init__(self):
        super().__init__()
        self.output_model_dir='model_0'
        self.log_txt = '%s/logs.txt' % 'model_0'
        if os.path.exists(self.log_txt):
            os.rename(self.log_txt, '%s_%s' % (self.log_txt, int(time.time())))
        with open(self.log_txt, 'a+') as f:
            f.write(' '.join(sys.argv))

    def on_epoch_end(self, epoch, logs=None):
        save_dir = '%s' % self.output_model_dir
        self.export_saved_model(save_dir, epoch)
        
        save_dir = '%s/%s.h5' % (self.output_model_dir, epoch)
        self.model.save(save_dir)
        print("model saved to to dir %s. %s" % (save_dir, logs))
        
        with open(self.log_txt, 'a+') as f:
            if epoch == 0:
                f.write('[%d] %s: %s\n' % (epoch, datetime.now().strftime('%Y-%m-%d %H:%M:%S'), logs))

    def export_saved_model(self, saved_dir, epoch):
        model_version = epoch
        model_signature = tf.saved_model.signature_def_utils.predict_signature_def(
            inputs={'input': self.model.input}, outputs={'output': self.model.output})
        export_path = os.path.join(compat.as_bytes(saved_dir), compat.as_bytes(str(model_version)))
        builder = tf.saved_model.builder.SavedModelBuilder(export_path)
        builder.add_meta_graph_and_variables(
            sess=K.get_session(),
            tags=[tf.saved_model.tag_constants.SERVING],
            clear_devices=True,
            signature_def_map={
                tf.saved_model.signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY:
                    model_signature
            })
        builder.save()   

In [4]:
#     def retrain(self, model_path, data, label, epoch):
#         model = self.load_model_fromfile(model_path)
#         print("load model, evaluate it.")
#         loss, accuracy = model.evaluate(data, label)
#         print("load model, loss = %s, acc =%s ." % (loss, accuracy))
#         self.params['batch_size']=124
#         model.fit(data, label, batch_size=self.params['batch_size'], callbacks=[ZpCallBack()], epochs=epoch)
# #         model.fit(data, label, batch_size=124, epochs=epoch)

#     def load_model_fromfile(self, model_path):
#         crf=CRF(self.params['Label_class'], sparse_target=True)
#         return load_model(model_path, custom_objects={"CRF": crf, 'crf_loss': crf.loss_function, 'crf_viterbi_accuracy': crf.accuracy})

#     def predict(self, model_path, data, maxlen):
#         model = self.model
#         char2id = [self.vocab.get(i) for i in data]
#         input_data = pad_sequences([char2id], maxlen)
#         model.load_weights(model_path)
#         result = model.predict(input_data)[0][-len(data):]
#         result_label = [np.argmax(i) for i in result]
#         return result_label

In [5]:
def get_evaluation_plot(history):
    fig1, ax_acc = plt.subplots()
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title('Model - Accuracy')
    plt.legend(['Training', 'Validation'], loc='lower right')
    plt.show()
    gc.collect()

    fig2, ax_loss = plt.subplots()
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Model- Loss')
    plt.legend(['Training', 'Validation'], loc='upper right')
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.show()
    gc.collect()

In [6]:
def build_model(params):
    model = Sequential()
    model.add(Embedding(params['Embedding_leng'], params['Embedding_dim'], mask_zero=True))  # Random embedding
    model.add(Bidirectional(LSTM(params['biLSTM_units'], return_sequences=True)))
    crf = CRF(params['Label_class'], sparse_target=True)
    model.add(crf)
    model.summary()
    model.compile('adam', loss=crf_loss, metrics=[crf_viterbi_accuracy]) #metrics=[crf.accuracy]
    return model

In [7]:
train_data_path='train_ner_3_bieo.csv'
test_data_path='test_ner_3_bieo.csv'
vocab_file_path='vocab_cosmetic_3_bieo.json'
labels = ['O','BRD-B','BRD-M','BRD-E']

w2d=Word2Id(vocab_file_path,train_data_path,labels)
vocab,label2id=w2d.load()

{'O': 0, 'BRD-B': 1, 'BRD-M': 2, 'BRD-E': 3}


In [29]:
train_x=np.load("train_x_3_bieo.npy")
print(train_x.shape)

train_y=np.load("train_y_3_bieo.npy")
print(train_y.shape)

(3027229, 50)
(3027229, 50, 1)


In [30]:
# """快速测save best model"""
# i=12345
# train_x=np.load("train_x_3_bieo.npy")[i:i+5000]
# print(train_x.shape)
# train_y=np.load("train_y_3_bieo.npy")[i:i+5000]
# print(train_y.shape)

In [8]:
char_pad_embedding=50

In [9]:
params={'Embedding_dim':200,
        'Embedding_leng':len(vocab),
        'biLSTM_units':100,
        'Label_class':len(labels),
        'total_epoch':30, #建议放30,到时候可以加一个early_stopping
        'batch_size':64}
print(params)

{'Embedding_dim': 200, 'Embedding_leng': 5331, 'biLSTM_units': 100, 'Label_class': 4, 'total_epoch': 30, 'batch_size': 64}


In [33]:
model=build_model(params)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 200)         1066200   
_________________________________________________________________
bidirectional_2 (Bidirection (None, None, 200)         240800    
_________________________________________________________________
crf_2 (CRF)                  (None, None, 4)           828       
Total params: 1,307,828
Trainable params: 1,307,828
Non-trainable params: 0
_________________________________________________________________


In [35]:
# kfold5折交叉训练
# params['n_split']=5

# kf=KFold(n_splits=params['n_split'])
# kf_list=list(kf.split(train_x,train_y))
# print(len(kf_list[0][0]))
# print(len(kf_list[0][1]))

# train_acc_list=[]
# valid_acc_list=[]
# train_loss_list=[]
# valid_loss_list=[]

# for i in range(len(kf_list)):
#     print('FOLD : ',i)
#     train_idx=kf_list[i][0]
#     valid_idx=kf_list[i][1]
#     train_x_batch,train_y_batch=train_x[train_idx],train_y[train_idx]
#     valid_x_batch,valid_y_batch=train_x[valid_idx],train_y[valid_idx]
#     print(train_x_batch.shape,train_y_batch.shape)
#     print(valid_x_batch.shape,valid_y_batch.shape)
    
#     file_name='%d_bieo_model_{epoch:02d}-{val_loss:.2f}.h5'%i
#     savepath="%s/%s"%(file_path,file_name)
#     print('the save file is :',savepath)

#     checkpoint=ModelCheckpoint(
#             filepath=savepath,
#             monitor='val_loss',
#             save_best_only=True,
#             verbose=1,
#             save_weights_only=True,
#             period=3)

#     history=model.fit(train_x_batch,train_y_batch,validation_data=(valid_x_batch,valid_y_batch),batch_size=params['batch_size'],
#                           callbacks=[checkpoint],epochs=params['total_epoch'],verbose=1)
#     get_evaluation_plot(history)
    
#     train_loss_batch,train_acc_batch=model.evaluate(train_x_batch,train_y_batch)
#     valid_loss_batch,valid_acc_batch=model.evaluate(valid_x_batch,valid_y_batch)
#     print('train loss is %f, valid loss is %f'%(train_loss_batch,valid_loss_batch))
#     print('train acc is %f, valid acc is %f'%(train_acc_batch,valid_acc_batch))
    
#     train_acc_list.append(train_acc_batch)
#     valid_acc_list.append(valid_acc_batch)
#     train_loss_list.append(train_loss_batch)
#     valid_acc_list.append(valid_loss_batch)
    
#     gc.collect()
#     print('-'*50)

# print('最终结果展示：')
# print(np.mean(train_acc_list))
# print(np.mean(valid_acc_list))
# print(np.mean(train_loss_list))
# print(np.mean(valid_loss_list))
# gc.collect()

In [36]:
train_xx,valid_xx,train_yy,valid_yy=train_test_split(train_x,train_y,test_size=0.15,random_state=2024)
print(train_xx.shape,train_yy.shape)
print(valid_xx.shape,valid_yy.shape)

(2573144, 50) (2573144, 50, 1)
(454085, 50) (454085, 50, 1)


In [38]:
tm=datetime.strftime(datetime.now(),'%Y%m%d')
print(tm)

file_path='model_%s'%tm

if not os.path.exists(file_path):
    os.mkdir(file_path)
    
file_name='model_{epoch:02d}-{val_loss:.2f}.h5'
savepath="%s/%s"%(file_path,file_name)
print(savepath)

20210224
model_20210224/model_{epoch:02d}-{val_loss:.2f}.h5


In [40]:
checkpoint=ModelCheckpoint(
        filepath=savepath,
        monitor='val_loss',
        save_best_only=True,
        verbose=1,
        save_weights_only=True,
        period=3)

In [ ]:
history=model.fit(train_xx,train_yy,validation_data=(valid_xx,valid_yy),batch_size=params['batch_size'],callbacks=[checkpoint],epochs=params['total_epoch'])

Train on 2573144 samples, validate on 454085 samples
Epoch 1/30
2573144/2573144 [==============================] - 12397s 5ms/step - loss: 0.8839 - crf_viterbi_accuracy: 0.9853 - val_loss: 0.8751 - val_crf_viterbi_accuracy: 0.9907
Epoch 2/30
2573144/2573144 [==============================] - 12383s 5ms/step - loss: 0.8743 - crf_viterbi_accuracy: 0.9919 - val_loss: 0.8740 - val_crf_viterbi_accuracy: 0.9923
Epoch 3/30
2251648/2573144 [=========================>....] - ETA: 23:45 - loss: 0.8736 - crf_viterbi_accuracy: 0.9929

### <font color=red>测试test_100的数据

In [11]:
q0=pd.read_csv('test_100_on_bio_bieo.csv')
print(q0.shape)

(1000, 9)


In [13]:
the_model=build_model(params)
save_path='./model_20210224/model_06-0.87.h5'
the_model.load_weights(save_path)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 200)         1066200   
_________________________________________________________________
bidirectional_2 (Bidirection (None, None, 200)         240800    
_________________________________________________________________
crf_2 (CRF)                  (None, None, 4)           828       
Total params: 1,307,828
Trainable params: 1,307,828
Non-trainable params: 0
_________________________________________________________________


In [14]:
print(label2id)

{'O': 0, 'BRD-B': 1, 'BRD-M': 2, 'BRD-E': 3}


In [15]:
def get_one_predict_label_post(data,res,pad_embedding):
    s1=len(data)
    s2=len(res)
    lst=[]
    
    if s1==s2:
        for i in range(s1):
            if res[i] == 'BRD-B' or res[i] == 'BRD-M' or res[i]=='BRD-E':
                lst.append(data[i])
    else:
        for i in range(s2):
            if res[i]=='BRD-B' or res[i]=='BRD-M'or res[i]=='BRD-E':
                lst.append(data[i])
                
    if len(lst)>=1:
        return ''.join(lst)
    else:
        return np.nan

def predict_one_post(model, data, pad_embedding):
    print(data)
    char2id = [vocab.get(i) for i in data]
    print(char2id)
    
    input_data = pad_sequences([char2id], pad_embedding, padding='post', truncating='post')
#     print(input_data)
    
    result =model.predict(input_data)[0]
#     print(result)
    
    result = result[:len(data)]
#     print(result)
    
    result_label = [np.argmax(i) for i in result]
    res2label=[labels[i] for i in result_label]
#     print(res2label)
    
    label=get_one_predict_label_post(data,res2label,pad_embedding)
    print(label)
    print('-'*30)
    return label

In [16]:
import re
def further_clean(v):
    v=str(v).replace(' ','')
    v=v.upper()
    pattern='[^0-9A-Z\u4e00-\u9fa5]'
    v=re.sub(pattern,'',v)
    return v

In [17]:
q0['desc']=q0['platform_goods_name'].apply(further_clean)

In [18]:
def raise_error(the_model,x,pad_embedding):
    try:
        return predict_one_post(the_model,x,pad_embedding)
    except:
        return 'unknown word'

In [19]:
q0['res']=q0['desc'].apply(lambda x:raise_error(the_model,x,char_pad_embedding))

COLORKEY珂拉琪小金筷眉笔持久防水防汗初学者超细女学生不易脱色
[5167, 4380, 995, 4380, 2892, 4835, 3225, 4236, 4243, 4405, 3642, 3963, 4060, 623, 4919, 1383, 3025, 3154, 2102, 1976, 2102, 4004, 2210, 2018, 3276, 1156, 3200, 3582, 2018, 4146, 1714, 3223, 4085, 5131]
COLORKEY
------------------------------
COLORKEY珂拉琪小黑镜唇釉镜面漆光唇彩唇蜜口红持久显白学生平价
[5167, 4380, 995, 4380, 2892, 4835, 3225, 4236, 4243, 4405, 3642, 3963, 994, 5308, 1861, 88, 5308, 5115, 2732, 4772, 1861, 2995, 1861, 104, 2228, 736, 3025, 3154, 1281, 3198, 2018, 4146, 3542, 2787]
COLORKEY
------------------------------
晶采艳色滋润口红持久锁水持妆唇彩丰润保湿显白不易掉色
[1197, 4120, 4757, 5131, 4844, 705, 2228, 736, 3025, 3154, 4037, 1976, 3025, 2948, 1861, 2995, 3248, 705, 3423, 2957, 1281, 3198, 1714, 3223, 4023, 5131]
nan
------------------------------
逐本森韵清欢晨蜜天然水感植物卸妆油敏感肌脸部深层清洁卸妆水膏
[4439, 3562, 3720, 425, 2660, 402, 1890, 104, 2478, 4389, 1976, 1900, 1254, 4215, 4239, 2948, 380, 4642, 1900, 751, 1601, 2129, 5185, 231, 2660, 658, 4239, 2948, 1976, 1037]
逐本
------------------------

BLANKME
------------------------------
韩国ROMAND果汁镜面唇釉12水膜20奶茶口红18哑光透明唇蜜新品06
[4096, 3348, 2892, 4380, 73, 3630, 3852, 3359, 561, 3222, 5308, 5115, 1861, 88, 597, 4705, 1976, 47, 4705, 3987, 2031, 2730, 2228, 736, 597, 2883, 875, 4772, 2591, 2624, 1861, 104, 5012, 526, 3987, 1096]
ROMAND
------------------------------
CAVILLA卡维拉睫毛增长液正品卡薇拉官网男女眉毛生长浓密睫毛膏
[5167, 3630, 3993, 1896, 995, 995, 3630, 2099, 2156, 4405, 4054, 5214, 1884, 2666, 2619, 5193, 526, 2099, 2983, 4405, 4566, 2731, 3232, 3582, 4919, 5214, 4146, 2666, 394, 4487, 4054, 5214, 1037]
CAVILLA
------------------------------
天使之眼小蓝油卸妆油天然水感植物水膏次抛敏感肌专用深层清洁
[2478, 4647, 2326, 4321, 3963, 4643, 380, 4239, 2948, 380, 2478, 4389, 1976, 1900, 1254, 4215, 1976, 1037, 489, 2266, 4642, 1900, 751, 3578, 2437, 5185, 231, 2660, 658]
天使之眼
------------------------------
VYFARMACY卸妆膏清洁膏100ML赠50ML一秒乳化温和眼唇卸
[3993, 4236, 2576, 3630, 2892, 73, 3630, 5167, 4236, 4239, 2948, 1037, 2660, 658, 1037, 597, 3987, 3987, 73, 995, 2859, 999, 3987, 73, 995, 2136

BLANKME
------------------------------
COLORCANDY糖果彩色钻石养肤粉底液控油持久遮瑕保湿混干皮不脱妆
[5167, 4380, 995, 4380, 2892, 5167, 3630, 3852, 3359, 4236, 730, 561, 2995, 5131, 4614, 5288, 4423, 4682, 1218, 1770, 2619, 3136, 380, 3025, 3154, 3969, 3932, 3423, 2957, 1513, 563, 268, 1714, 4085, 2948]
COLORCANDY
------------------------------
入店必买BLANKME小银盒气垫BB霜粉底霜持久不脱妆混油皮
[2914, 187, 1696, 2370, 2866, 995, 3630, 3852, 4835, 73, 3225, 3963, 1664, 2658, 2146, 1494, 2866, 2866, 1903, 1218, 1770, 1903, 3025, 3154, 1714, 4085, 2948, 1513, 380, 268]
BLANKME
------------------------------
ANGELEYES天使之眼持妆色粉底液哑光控油遮瑕保湿持久混合油干皮
[3630, 3852, 4460, 3225, 995, 3225, 4236, 3225, 4838, 2478, 4647, 2326, 4321, 3025, 2948, 5131, 1218, 1770, 2619, 875, 4772, 3136, 380, 3969, 3932, 3423, 2957, 3025, 3154, 1513, 4885, 380, 563, 268]
ANGELEYES
------------------------------
抖音爆卖YERS眼睫毛增长液膏自然纤长浓密卷翘生长眉毛
[4832, 3490, 1780, 2763, 4236, 3225, 2892, 4838, 4321, 4054, 5214, 1884, 2666, 2619, 1037, 727, 4389, 1787, 2666, 394, 4487, 4557,

花漾
------------------------------
COLORKEY珂拉琪小黑镜唇釉孟美岐镜面漆光唇彩唇蜜口红持久显白
[5167, 4380, 995, 4380, 2892, 4835, 3225, 4236, 4243, 4405, 3642, 3963, 994, 5308, 1861, 88, 228, 1424, 4544, 5308, 5115, 2732, 4772, 1861, 2995, 1861, 104, 2228, 736, 3025, 3154, 1281, 3198]
COLORKEY
------------------------------
现货INTOYOU唇泥08雾面丝绒LNTO唇釉12口红05INTIYOU女YUOINTOU
[2933, 108, 1896, 3852, 3314, 4380, 4236, 4380, 2425, 1861, 2910, 3987, 2883, 3228, 5115, 3698, 5057, 995, 3852, 3314, 4380, 1861, 88, 597, 4705, 2228, 736, 3987, 999, 1896, 3852, 3314, 1896, 4236, 4380, 2425, 3582, 4236, 2425, 4380, 1896, 3852, 3314, 4380, 2425]
INTOYOUYUOINTOU
------------------------------
德国JIMOK电动牙刷全自动充电式美白男女士成人超声波情侣套装
[209, 3348, 3898, 1896, 73, 4380, 4835, 3514, 3288, 4021, 4254, 4272, 727, 3288, 2195, 3514, 2503, 1424, 3198, 3232, 3582, 3018, 3919, 2212, 1156, 3561, 659, 1897, 2368, 2122, 3377]
JIMOK
------------------------------
新年礼物ROMAND雾面哑光丝绒天鹅绒唇釉彩口红女砖红保税仓
[5012, 1669, 2100, 4215, 2892, 4380, 73, 3630, 3852, 3359, 32

nan
------------------------------
GOGOTALES戈戈舞小白方高光修容盘闪粉脸部提亮神仙土豆泥偏光
[4460, 4380, 4460, 4380, 3314, 3630, 995, 3225, 4838, 2990, 2990, 2121, 3963, 3198, 1053, 5105, 4772, 3329, 1816, 1694, 5327, 1218, 1601, 2129, 3184, 3024, 4047, 2007, 1964, 3361, 2910, 4912, 4772]
GOGOTALES
------------------------------
郑瑄茉气垫遮瑕保湿控油持久不脱妆气垫BB霜粉扑替换装官方旗舰店
[2819, 987, 2044, 2146, 1494, 3969, 3932, 3423, 2957, 3136, 380, 3025, 3154, 1714, 4085, 2948, 2146, 1494, 2866, 2866, 1903, 1218, 2532, 3846, 3270, 3377, 4566, 1053, 651, 1750, 187]
郑瑄茉
------------------------------
美国RCMA黑胡椒粉散粉定妆粉蜜粉无色控油持久官方旗舰店正品女
[1424, 3348, 2892, 5167, 73, 3630, 994, 3135, 3944, 1218, 3737, 1218, 3017, 2948, 1218, 104, 1218, 3458, 5131, 3136, 380, 3025, 3154, 4566, 1053, 651, 1750, 187, 5193, 526, 3582]
RCMA
------------------------------
APINKBABY三色六色眼影盘腮红
[3630, 4208, 1896, 3852, 4835, 2866, 3630, 2866, 4236, 4859, 5131, 1835, 5131, 4321, 5055, 1694, 1165, 736]
APINKBABY
------------------------------
HYNTOOR黑兎黑兔唇泥马卡龙哑光雾面女学生平价丝绒

...

nan
------------------------------
李佳埼化妆棉卸妆棉盒装卸妆用脸部湿敷专用巾拍爽肤水纯棉片厚款
[3605, 825, 904, 1938, 2948, 2847, 4239, 2948, 2847, 2658, 3377, 4239, 2948, 2437, 1601, 2129, 2957, 1029, 3578, 2437, 1926, 381, 1248, 4682, 1976, 248, 2847, 2116, 2217, 4839]
nan
------------------------------
AKF睫毛膏女防水纤长卷翘持久不晕染细刷头极细加长官方旗舰店
[3630, 4835, 2576, 4054, 5214, 1037, 3582, 2102, 1976, 1787, 2666, 4557, 4506, 3025, 3154, 1714, 2145, 2367, 3200, 4254, 4474, 36, 3200, 975, 2666, 4566, 1053, 651, 1750, 187]
AKF
------------------------------
初学者彩妆套装化妆品全套组合美妆套盒女淡妆新手一整套学生正品
[2210, 2018, 3276, 2995, 2948, 2122, 3377, 1938, 2948, 526, 4272, 2122, 5224, 4885, 1424, 2948, 2122, 2658, 3582, 3899, 2948, 5012, 4929, 2136, 1889, 2122, 2018, 4146, 5193, 526]
nan
------------------------------
BIBIDIDI小方块口红雾面丝绒哑光唇膏持久保湿显白不掉色大牌正品
[2866, 1896, 2866, 1896, 3359, 1896, 3359, 1896, 3963, 1053, 384, 2228, 736, 3228, 5115, 3698, 5057, 875, 4772, 1861, 1037, 3025, 3154, 3423, 2957, 1281, 3198, 1714, 4023, 5131, 3298, 4310, 5193, 526]

SALMON鲑鱼
------------------------------
TF口红小样试用装大牌正品1680405迷你196黑管专柜999滋润唇膏
[3314, 2576, 2228, 736, 3963, 3505, 1070, 2437, 3377, 3298, 4310, 5193, 526, 597, 1096, 2883, 3987, 3272, 3987, 999, 1426, 2344, 597, 607, 1096, 994, 1869, 3578, 3262, 607, 607, 607, 4844, 705, 1861, 1037]
TF
------------------------------
MOONEYES花间酒镜面唇釉唇彩唇蜜口红持久滋润网红玻璃唇学生平价
[73, 4380, 4380, 3852, 3225, 4236, 3225, 4838, 1131, 1765, 4653, 5308, 5115, 1861, 88, 1861, 2995, 1861, 104, 2228, 736, 3025, 3154, 4844, 705, 2731, 736, 4923, 721, 1861, 2018, 4146, 3542, 2787]
MOONEYES
------------------------------
10支化妆刷套装眼影刷美妆刷子化妆超柔软散粉腮红平价粉底刷全套
[597, 3987, 2901, 1938, 2948, 4254, 2122, 3377, 4321, 5055, 4254, 1424, 2948, 4254, 4035, 1938, 2948, 1156, 1366, 4484, 3737, 1218, 1165, 736, 3542, 2787, 1218, 1770, 4254, 4272, 2122]
nan
------------------------------
OSELLINE欧采莲轻纱绒雾唇釉口红女学生雾面哑光平价唇彩唇蜜持久
[4380, 4838, 3225, 995, 995, 1896, 3852, 3225, 2010, 4120, 731, 4134, 4112, 5057, 3228, 1861, 88, 2228, 736, 3582, 2018, 4146

AUBURN
------------------------------
歌诗美指甲油女无毒无味孕妇免烤快干持久裸2020年秋冬新色系透明
[4274, 5130, 1424, 2736, 1648, 380, 3582, 3458, 2678, 3458, 4938, 251, 1645, 4302, 964, 4081, 563, 3025, 3154, 3746, 4705, 3987, 4705, 3987, 1669, 2932, 234, 5012, 5131, 3918, 2591, 2624]
歌诗美
------------------------------
PRAMY柏瑞美散粉持久控油胡椒粉定妆粉博瑞美防水不脱妆平价蜜粉
[4208, 2892, 3630, 73, 4236, 4720, 300, 1424, 3737, 1218, 3025, 3154, 3136, 380, 3135, 3944, 1218, 3017, 2948, 1218, 3810, 300, 1424, 2102, 1976, 1714, 4085, 2948, 3542, 2787, 104, 1218]
PRAMY柏瑞美
------------------------------
现货JCJOOCYEE酵色唇釉X乐乐琥珀丝绒镜面脏脏奶咖0678V04V01
[2933, 108, 3898, 5167, 3898, 4380, 4380, 5167, 4236, 3225, 3225, 850, 5131, 1861, 88, 6, 3872, 3872, 3392, 441, 3698, 5057, 5308, 5115, 2247, 2247, 2031, 3459, 3987, 1096, 4961, 2883, 3993, 3987, 3272, 3993, 3987, 597]
JOOCYEE
------------------------------
芭集粉饼定妆粉干粉控油久持遮瑕耐水防汗不易脱妆女八集
[4258, 5189, 1218, 3874, 3017, 2948, 1218, 563, 1218, 3136, 380, 3154, 3025, 3969, 3932, 5238, 1976, 2102, 4004, 1714, 32

nan
------------------------------
菲丝加芬松毯化妆刷4支眼部套刷柔软提纯细光锋眼影刷美妆工具
[460, 3698, 975, 3628, 4702, 2132, 1938, 2948, 4254, 3272, 2901, 4321, 2129, 2122, 4254, 1366, 4484, 3184, 248, 3200, 4772, 4414, 4321, 5055, 4254, 1424, 2948, 3412, 2786]
菲丝加芬
------------------------------
韩国MEDICUBE乐得粉底液遮瑕持久控油轻薄干皮毫米小样旗舰店官方
[4096, 3348, 73, 3225, 3359, 1896, 5167, 2425, 2866, 3225, 3872, 1419, 1218, 1770, 2619, 3969, 3932, 3025, 3154, 3136, 380, 4134, 2286, 563, 268, 285, 4687, 3963, 3505, 651, 1750, 187, 4566, 1053]
MEDICUBE
------------------------------
ROMAND唇釉果汁20奶油杏色18新色19镜面水光平价口红小众品牌学生
[2892, 4380, 73, 3630, 3852, 3359, 1861, 88, 561, 3222, 4705, 3987, 2031, 380, 934, 5131, 597, 2883, 5012, 5131, 597, 607, 5308, 5115, 1976, 4772, 3542, 2787, 2228, 736, 3963, 734, 526, 4310, 2018, 4146]
ROMAND
------------------------------
CHARM诱惑小黑瓶胶原蛋白粉底液轻薄遮瑕保湿控油持久自然不脱妆
[5167, 3453, 3630, 2892, 73, 4475, 3344, 3963, 994, 5276, 3990, 536, 1109, 3198, 1218, 1770, 2619, 4134, 2286, 3969, 3932, 3423, 2957, 3136, 38

nan
------------------------------
5包1000片FAKEFACE化妆棉女卸妆棉卸妆用脸部厚款纯棉省水湿敷
[999, 75, 597, 3987, 3987, 3987, 2116, 2576, 3630, 4835, 3225, 2576, 3630, 5167, 3225, 1938, 2948, 2847, 3582, 4239, 2948, 2847, 4239, 2948, 2437, 1601, 2129, 2217, 4839, 248, 2847, 2264, 1976, 2957, 1029]
FAKEFACE
------------------------------
专业化妆刷眼影遮瑕粉底腮红散粉高光眼线鼻影眉唇刷子套装超柔软
[3578, 1242, 1938, 2948, 4254, 4321, 5055, 3969, 3932, 1218, 1770, 1165, 736, 3737, 1218, 5105, 4772, 4321, 2747, 132, 5055, 4919, 1861, 4254, 4035, 2122, 3377, 1156, 1366, 4484]
nan
------------------------------
MOROVANPOLY凝胶套装延伸凝胶套件增强凝胶套件基础顶涂层一体化技术人员套装4色聚酯套装
[73, 4380, 2892, 4380, 3993, 3630, 3852, 4208, 4380, 995, 4236, 2677, 3990, 2122, 3377, 3757, 2557, 2677, 3990, 2122, 1470, 1884, 2488, 2677, 3990, 2122, 1470, 155, 1805, 1535, 4100, 231, 2136, 1255, 1938, 2186, 5234, 2212, 91, 2122, 3377, 3272, 5131, 1879, 2900, 2122, 3377]
MOROVAN
------------------------------
YNM睫毛膏双头女防水纤长卷翘不晕染不脱妆浓密官方旗舰店官YMN
[4236, 3852, 73, 4054, 5214, 1037, 3007, 4

In [20]:
q0.rename(columns={'res':'bieo_res'},inplace=True)

In [21]:
q0.to_csv('test_100_bieo.csv',index=False,encoding='utf-8-sig')
gc.collect()

7172